In [26]:
from transformers import TFBertForSequenceClassification, BertTokenizer
# import torch
import tensorflow as tf


In [27]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# def predict_similarity(text1, text2):
#     # Tokenize the texts
#     encoded_dict1 = tokenizer.encode_plus(text1, add_special_tokens=True, 
#                                            max_length=64, padding='max_length', 
#                                            return_attention_mask=True, return_tensors='pt')
#     encoded_dict2 = tokenizer.encode_plus(text2, add_special_tokens=True, 
#                                            max_length=64, padding='max_length', 
#                                            return_attention_mask=True, return_tensors='pt')
#     # Get the model's prediction
#     with torch.no_grad():
#         logits = model(encoded_dict1['input_ids'], attention_mask=encoded_dict1['attention_mask'], 
#                        token_type_ids=None)[0]
#         pred1 = torch.softmax(logits, dim=1)[0][1].item()
#         logits = model(encoded_dict2['input_ids'], attention_mask=encoded_dict2['attention_mask'], 
#                        token_type_ids=None)[0]
#         pred2 = torch.softmax(logits, dim=1)[0][1].item()
#     # Return the similarity score
#     return max(pred1, pred2)


In [29]:
def predict_similarity(text1, text2):
    # Tokenize the texts
    encoded_dict1 = tokenizer.encode_plus(text1, add_special_tokens=True, 
                                           max_length=64, padding='max_length', 
                                           return_attention_mask=True, return_tensors='tf')
    encoded_dict2 = tokenizer.encode_plus(text2, add_special_tokens=True, 
                                           max_length=64, padding='max_length', 
                                           return_attention_mask=True, return_tensors='tf')
    # Get the model's prediction
    logits1 = model(encoded_dict1['input_ids'], attention_mask=encoded_dict1['attention_mask'], 
                    token_type_ids=None)[0]
    pred1 = tf.nn.softmax(logits1, axis=1).numpy()[0][1]
    logits2 = model(encoded_dict2['input_ids'], attention_mask=encoded_dict2['attention_mask'], 
                    token_type_ids=None)[0]
    pred2 = tf.nn.softmax(logits2, axis=1).numpy()[0][1]
    # Return the similarity score
    return max(pred1, pred2)


In [37]:
raw = "Two women who just had lunch hugging and saying goodbye.	There are two woman in this picture.	1"
text1 = "The quick brown fox jumps over the lazy dog."
text2 = "The lazy dog is jumped over by the quick brown fox."
text1, text2, label = raw.split('\t')
similarity = predict_similarity(text1, text2)
print(similarity)


0.33824262


In [39]:
def check(raw):
	text1, text2, label = raw.split('\t')
	similarity = predict_similarity(text1, text2)
	return similarity, label

In [42]:
check("A man with blond-hair, and a brown shirt drinking out of a public water fountain.	A blond man wearing a brown shirt is reading a book on a bench in the park	0")

(0.36488363, '0')

In [44]:
check("A man with blond-hair, and a brown shirt drinking out of a public water fountain.	A blond man drinking water from a fountain.	1")

(0.29347184, '1')